## DuckDB Notebook

This notebook generates a bunch of raw outputs, without applying PAC, to be consumed by a second stage.

```
 group by              output cols                        
 key cols ┌────────┬────────┬────────┬────────┐           
        │ │   A    │   B    │   C    │   D    │           
      ┌─▼─┼────────┼────────┼────────┼────────┤           
      │ 1 │   2    │        │        │        │           
      ├───┼───|────┼────────┼────────┼────────┤           
      │ 2 │   │    │        │        │        │           
      ├───┼───┼────┼────────┼────────┼────────┤           
      │ 3 │   │    │        │        │        │           
      └───┴───┼────┴────────┴────────┴────────┘           
              ▼                 A_1.json                  
       Sample 0:   A1=2        ┌─────────────────────────┐
       Sample 1:   A1=4  ───▶  │{                        │
             ...               │    col: A               │
       Sample 999: A1=3        │    row: 1               │
                               │    value: [2, 4, ... 3] │
                               │}                        │
                               └─────────────────────────┘
```

In [1]:
#!/usr/bin/env python
# coding: utf-8

EXPERIMENT = 'pac-duckdb-q1'
OUTPUT_DIR = f'./outputs/{EXPERIMENT}'
GENERATE = False
USE_EVEN_NUMBER_OF_INPUT_ROWS = False

if GENERATE:
    print("GENERATE = True, so we will generate new samples.")
else:
    print("GENERATE = False, so we will load saved output from files rather than recomputing.")

import os
from typing import List
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

import numpy as np
import pickle

import duckdb
import polars as pl
import pyarrow as pa

# duckdb load data/tpch/tpch.duckdb
#con = duckdb.connect(database='data/tpch/tpch.duckdb', read_only=True)
con = duckdb.connect(database=':memory:')
tables = ["customer", "lineitem", "nation", "orders", "part", "partsupp", "region", "supplier"]
#tables = ["lineitem", "orders"]
for t in tables:
    con.execute(f"CREATE TABLE {t} AS SELECT * FROM 'data/tpch/{t}.parquet'")

lineitem_df = con.execute("SELECT * FROM lineitem").fetchdf()
orders_df = con.execute("SELECT * FROM orders").fetchdf()

row_count = lineitem_df.shape[0]

GENERATE = False, so we will load saved output from files rather than recomputing.


In [2]:
# Construct the table of random samples
# to use, join it with the lineitem table (for specific sample # s) and filter to just the
# rows where random_binary = 1.0
# This will give us a 50% sample of the lineitem table for each sample # s

SAMPLES = 1024
assert SAMPLES % 2 == 0, "SAMPLES must be even to create complementary samples."

random_samples = con.execute(f"""
DROP TABLE IF EXISTS random_samples;

CREATE TABLE random_samples AS
WITH sample_numbers AS MATERIALIZED (
    SELECT range AS sample_id FROM range({SAMPLES//2})
), random_values AS MATERIALIZED (
    SELECT 
        sample_numbers.sample_id,
        customer.rowid AS row_id,
        (RANDOM() > 0.5)::BOOLEAN AS random_binary
    FROM sample_numbers
    JOIN customer ON TRUE  -- Cross join to duplicate rows for each sample
)
SELECT
    sample_id,
    row_id,
    random_binary
FROM random_values
UNION ALL
SELECT -- select the complementary samples too
    ({SAMPLES//2}) + sample_id,
    row_id,
    NOT random_binary  -- Inverse the random_binary to get the complementary sample
FROM random_values
ORDER BY sample_id, row_id;
""")

The randomness of what rows are chosen is saved to disk in `random_binary.json`. For each sample #, there is an array with one entry per row, where 1 means the row was chosen and 0 means it was not.

In [3]:
con.execute(f"""
SELECT sample_id, array_agg(random_binary::TINYINT) as random_binary
FROM random_samples
GROUP BY sample_id;
""").pl().write_json(f"{OUTPUT_DIR}/random_binary.json")

Query is specified as a prepared statement. We will then execute it once per sample.

In [4]:
# Query
con.execute("""
DEALLOCATE PREPARE run_query;

PREPARE run_query AS 
SELECT
    l_returnflag,
    l_linestatus,
    2*sum(l_quantity) AS sum_qty,
    2*sum(l_extendedprice) AS sum_base_price,
    2*sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    2*sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    2*count(*) AS count_order
FROM
    lineitem
JOIN orders ON lineitem.l_orderkey = orders.o_orderkey
JOIN customer ON orders.o_custkey = customer.c_custkey
JOIN random_samples AS rs
    ON rs.row_id = customer.rowid
WHERE
    l_shipdate <= CAST('1998-09-02' AS date)
    AND rs.random_binary = TRUE
    AND rs.sample_id = $sample
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
""")

# Run query to see output
dfs0 = con.execute(f"EXECUTE run_query(sample := {0});").pl()

# Save csv copies of the first 5 samples
os.makedirs(f"{OUTPUT_DIR}/csv", exist_ok=True)
for s in range(5):
    con.execute(f"EXECUTE run_query(sample := {s});").pl().write_csv(f"{OUTPUT_DIR}/csv/sample_{s}.csv")

dfs0

l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
str,str,"decimal[38,2]","decimal[38,2]","decimal[38,4]","decimal[38,6]",f64,f64,f64,i64
"""A""","""F""",3731942.00,5260801969.08,4997446538.6702,5197033283.188786,25.511621,35962.934901,0.050055,146284
"""N""","""F""",95700.00,134642322.14,127937683.9852,132993628.770162,25.656836,36097.137303,0.05008,3730
"""N""","""O""",7462590.00,10517402350.66,9991139857.0666,10390947239.921324,25.58538,36058.758582,0.050081,291674
"""R""","""F""",3780618.00,5329893603.86,5064366563.7992,5267179596.870270,25.618109,36116.262833,0.049992,147576


In [5]:
# Run the query for each sample, but accumulate in a pl.DataFrame instead of a list
dfsdf: pl.DataFrame = pl.concat(
    con.execute(f"EXECUTE run_query(sample := {s});").pl().insert_column(0, pl.lit(s).alias("sample"))
    for s in range(SAMPLES)
)
dfsdf

sample,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
i32,str,str,"decimal[38,2]","decimal[38,2]","decimal[38,4]","decimal[38,6]",f64,f64,f64,i64
0,"""A""","""F""",3731942.00,5260801969.08,4997446538.6702,5197033283.188786,25.511621,35962.934901,0.050055,146284
0,"""N""","""F""",95700.00,134642322.14,127937683.9852,132993628.770162,25.656836,36097.137303,0.05008,3730
0,"""N""","""O""",7462590.00,10517402350.66,9991139857.0666,10390947239.921324,25.58538,36058.758582,0.050081,291674
0,"""R""","""F""",3780618.00,5329893603.86,5064366563.7992,5267179596.870270,25.618109,36116.262833,0.049992,147576
1,"""A""","""F""",3816524.00,5377533393.86,5107753730.5576,5312870068.079760,25.605663,36078.721193,0.050127,149050
…,…,…,…,…,…,…,…,…,…,…
1022,"""R""","""F""",3798856.00,5355572761.00,5088597488.9480,5292352124.457692,25.566027,36042.619026,0.050034,148590
1023,"""A""","""F""",3753886.00,5286557250.72,5021404001.8052,5222032850.621024,25.475982,35877.551752,0.050188,147350
1023,"""N""","""F""",95032.00,133208492.02,126432535.3078,131540697.600438,25.450455,35674.475635,0.050295,3734


In [6]:
# Define which columns are the group-by keys (INDEX_COLS) and which are the output columns (OUTPUT_COLS)
INDEX_COLS = ['l_returnflag', 'l_linestatus']
OUTPUT_COLS = ['sum_qty', 'sum_base_price', 'sum_disc_price', 'sum_charge', 'avg_qty', 'avg_price', 'avg_disc', 'count_order']

In [7]:
# Combine all the samples into one table, grouped-by the group-by keys. Each cell contains an n <= # of samples length array of values.
listdf = (dfsdf.group_by(INDEX_COLS, maintain_order=True)
      .all()
      .drop(pl.col("sample")))
listdf

l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
str,str,"list[decimal[38,2]]","list[decimal[38,2]]","list[decimal[38,4]]","list[decimal[38,6]]",list[f64],list[f64],list[f64],list[i64]
"""A""","""F""","[3731942.00, 3816524.00, … 3753886.00]","[5260801969.08, 5377533393.86, … 5286557250.72]","[4997446538.6702, 5107753730.5576, … 5021404001.8052]","[5197033283.188786, 5312870068.079760, … 5222032850.621024]","[25.511621, 25.605663, … 25.475982]","[35962.934901, 36078.721193, … 35877.551752]","[0.050055, 0.050127, … 0.050188]","[146284, 149050, … 147350]"
"""N""","""F""","[95700.00, 94734.00, … 95032.00]","[134642322.14, 132731005.26, … 133208492.02]","[127937683.9852, 126162520.6870, … 126432535.3078]","[132993628.770162, 131319950.371608, … 131540697.600438]","[25.656836, 25.115058, … 25.450455]","[36097.137303, 35188.495562, … 35674.475635]","[0.05008, 0.049634, … 0.050295]","[3730, 3772, … 3734]"
"""N""","""O""","[7462590.00, 7464586.00, … 7379966.00]","[10517402350.66, 10526203143.62, … 10406399145.52]","[9991139857.0666, 9999366391.3816, … 9885230825.2728]","[10390947239.921324, 10399319144.378976, … 10281054598.466496]","[25.58538, 25.539161, … 25.521026]","[36058.758582, 36014.106828, … 35986.883742]","[0.050081, 0.050096, … 0.050114]","[291674, 292280, … 289172]"
"""R""","""F""","[3780618.00, 3799432.00, … 3778702.00]","[5329893603.86, 5356468676.78, … 5326121166.36]","[5064366563.7992, 5089492623.2920, … 5060705391.1358]","[5267179596.870270, 5292357097.436320, … 5263046989.273994]","[25.618109, 25.516326, … 25.523148]","[36116.262833, 35973.114376, … 35975.151411]","[0.049992, 0.049978, … 0.049983]","[147576, 148902, … 148050]"


In [8]:
allgroups: pl.DataFrame = listdf.select(INDEX_COLS)
allgroups.to_dicts()

[{'l_returnflag': 'A', 'l_linestatus': 'F'},
 {'l_returnflag': 'N', 'l_linestatus': 'F'},
 {'l_returnflag': 'N', 'l_linestatus': 'O'},
 {'l_returnflag': 'R', 'l_linestatus': 'F'}]

In [9]:
# Template for the final output, including all possible group-by groups
# Obtained by collecting all the samples in a big table and then keeping only the first occurrence of each groupby key.
# Then, fill all OUTPUT_COLS with nulls
templatedf = dfsdf.drop(pl.col("sample")).group_by(INDEX_COLS, maintain_order=True).first()
templatedf = templatedf.clear(n=len(allgroups)).with_columns(allgroups)
templatedf

l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
str,str,"decimal[38,2]","decimal[38,2]","decimal[38,4]","decimal[38,6]",f64,f64,f64,i64
"""A""","""F""",null,null,null,null,null,null,null,null
"""N""","""F""",null,null,null,null,null,null,null,null
"""N""","""O""",null,null,null,null,null,null,null,null
"""R""","""F""",null,null,null,null,null,null,null,null


In [10]:
os.makedirs(f"{OUTPUT_DIR}/json", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/pkl", exist_ok=True)
i: int = 0
reverse_map = {}
for col in OUTPUT_COLS:
    for group in allgroups.iter_rows(named=True):
        values = listdf.filter(pl.col(k).eq(v) for k, v in group.items()).select(col).to_series()
        j = pl.DataFrame().with_columns([
            pl.lit(col).alias("col"),
            pl.lit(group).alias("row"),
            pl.lit(values).alias("values"),
        ])
        reverse_map[i] = (col, group)
        i+=1
        j.write_json(f"{OUTPUT_DIR}/json/{i}.json")
        # save pickle of numpy conversion of values
        pickle.dump(values.to_numpy(), open(f"{OUTPUT_DIR}/pkl/{i}.pkl", "wb"))
import json
with open(f"{OUTPUT_DIR}/reverse_map.json", "w") as f:
    json.dump(reverse_map, f)


In [11]:
def numpyify(df: pl.DataFrame) -> np.ndarray:
    return df.select(OUTPUT_COLS).to_numpy().flatten()

def tablify(arr: np.ndarray) -> pl.DataFrame:
    global OUTPUT_SHAPE, OUTPUT_SCHEMA
    return templatedf.update( # put values back into the original dataframe
        pl.DataFrame(
            arr.reshape(OUTPUT_SHAPE), # reshape to the original shape
            schema=OUTPUT_SCHEMA # coerce numpy array to the correct schema
        ) # index cols will be left unchanged (not updated b/c we only update output_cols)
    )

In [12]:
# zip the OUTPUT_DIR
import shutil
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)

'/Users/michael/projects/dpdb/pacdb/outputs/pac-duckdb-q1.zip'